# Exploring the Docking Results
In this notebook, we will explore the docking results we generated in the docking notebook.

In [1]:
import nglview as nv

import MDAnalysis as mda

pdb_id = "2zq2"

u = mda.Universe(f"pdb/pdb{pdb_id}.ent")

protein = u.select_atoms("protein")
ligand_A = u.select_atoms("resname 13U and altLoc A")
ligand_B = u.select_atoms("resname 13U and altLoc B")

In [3]:
view = nv.show_mdanalysis(protein)
view.clear_representations()
view.add_representation("surface", colorScheme="hydrophobicity")
view.add_component(ligand_A)
view

NGLWidget()

In [4]:
view = nv.show_mdanalysis(protein)
view.clear_representations()
view.add_representation("surface", colorScheme="hydrophobicity")
view.add_component(ligand_B)
view

NGLWidget()

In [7]:
ligand_A


<AtomGroup with 30 atoms>

In [8]:
ligand_B

<AtomGroup with 30 atoms>